## IMport

In [2]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output

In [7]:
expiry_sp = 'BANKNIFTY23504'
inst = pd.DataFrame(kite.instruments("NFO"))

expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [11]:
def buy_sell_order(strike_price = None,ce_pe = None, price = None, trans_type = None,order_type = "MARKET",quantity = 25 ):
    print(strike_price, ce_pe, price,trans_type, order_type)
    order = kite.place_order(
        variety = kite.VARIETY_REGULAR,
        exchange = kite.EXCHANGE_NFO,
        tradingsymbol = f"BANKNIFTY23504{strike_price}{ce_pe}",
        transaction_type = trans_type,
        quantity = quantity,
        product = kite.PRODUCT_MIS,
        order_type = order_type,
        price=price,
        validity=kite.VALIDITY_DAY,
        disclosed_quantity=None,
        trigger_price=None,
        squareoff=None,
        stoploss=None,
        trailing_stoploss=None,
        # tag="TradeviaPython",
    )

In [ ]:
# price consolidation i
from IPython.display import clear_output
import time
strike_price = 42900
position = 0
market  = False

lower_side = 42811
upper_side = 42911

target_lower = 49000
target_upper = 42488

sl_lower = 42228
sl_upper = 42299


if market:
    order_type  = kite.ORDER_TYPE_MARKET
else:
    order_type  = kite.ORDER_TYPE_LIMIT

symbol_ce = f"NFO:BANKNIFTY23504{strike_price}CE"
symbol_pe = f"NFO:BANKNIFTY23504{strike_price}PE"
# price = 385
while(True):
    bn_index = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
    op_begin_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
    op_begin_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
    print(bn_index,op_begin_ce,op_begin_pe)
    # buy 
    if bn_index > upper_side and position == 0:
        print('buy_up', bn_index) 
        trans_type = kite.TRANSACTION_TYPE_BUY
        op_begin_ce -= 5
        buy_sell_order(strike_price = strike_price,price =op_begin_ce, ce_pe = 'CE', trans_type = trans_type,order_type =order_type)
        position = 1 
        
    elif bn_index < lower_side and position == 0:
        print("buy_down",bn_index)
        trans_type = kite.TRANSACTION_TYPE_BUY
        op_begin_pe -= 5
        buy_sell_order(strike_price = strike_price,ce_pe = 'PE', trans_type = trans_type,order_type = order_type)
        position = 1
    
    # if position == 1 and (bn_index > target_upper or bn_index < sl_upper): 
    #     trans_type = kite.TRANSACTION_TYPE_SELL
    #     order_type  = kite.ORDER_TYPE_MARKET
    #     buy_sell_order(strike_price = strike_price,ce_pe = 'CE',price = None, trans_type = trans_type,order_type = order_type)
    #     break
    # elif position == 1 and (bn_index < target_lower or bn_index > sl_lower):
    #     trans_type = kite.TRANSACTION_TYPE_SELL
    #     buy_sell_order(strike_price = strike_price,ce_pe = 'PE',price = None, trans_type = trans_type,order_type = order_type)
    #     break
        
    time.sleep(5)
    clear_output(wait=True)
    # break

    

## 21 EMA

In [8]:
from util_fun import get_data_parllel
import talib

today_date = '2023-01-01'
start_dt = '2023-07-01'
end_dt = '2023-08-09'
time_frame = 'minute'
instument = 260105
df_day = get_data_parllel(kite, instument, time_frame , start_dt,end_dt)

df_day['ema_21'] = talib.EMA(df_day['Close'], timeperiod = 21)
df_day.dropna(inplace=True)
df_day['distance'] = (df_day['Close'] - df_day['ema_21'])
df_day = df_day.between_time('09:20', '03:20')
df_day.reset_index(inplace=True)


up_down = None
position_up = 0
position_down = 0
open_pos_up = 0
close_pos = 0
date_open = 0
sl = 0
df_all  = pd.DataFrame()
for r,c in df_day.iterrows():
    if  c.distance> 0:
        up_down = 1
    else:
        up_down = -1
    # upside
    if c.distance < -50 and up_down == -1 and position_up == 0:
        # print(c.Date,c.Close,'upside')
        date_open  = c.Date
        open_pos = c.Close
        position_up = 1 
        sl = c.Close  - 30

    # downside 
    elif c.distance > 50 and up_down == 1 and position_down == 0:
        # print(c.Date,c.Close, 'downside')
        date_open  = c.Date
        open_pos = c.Close
        position_down = 1 
        sl = c.Close + 30

    # sq of up upside 
    if position_up == 1 and (c.Close >= c.ema_21 or c.Close <= sl):
        position_up = 0
        close_pos = c.Close
        df = pd.DataFrame({'date_open': date_open, 'Date': c.Date, 'open':open_pos, 'close': close_pos, 'type':'upside'}, index  = [0] )
        df_all = df_all.append(df)
        # print('sqoff upmove',c.Date,c.Close)

    # sq down side 
    if position_down == 1 and (c.Close <= c.ema_21 or c.Close >= sl):
        position_down = 0
        close_pos = c.Close
        df = pd.DataFrame({'date_open': date_open,'Date': c.Date,'open':open_pos, 'close': close_pos,'type':'downside'}, index = [0] )
        df_all = df_all.append(df)
        # print('sqoff down move ',c.Date,c.Close)


df_all = df_all.reset_index(drop = True)
df_all['points'] = np.where(df_all.type == 'upside', df_all['close'] -df_all['open'],df_all['open'] -df_all['close'] )
df_all['date_only'] = df_all.Date.dt.date

df_all.groupby('date_only')["points"].sum()

Pandas Apply:   0%|          | 0/2 [00:00<?, ?it/s]

date_only
2023-07-03    -90.00
2023-07-04   -414.20
2023-07-05     94.75
2023-07-06    212.65
2023-07-07    -18.85
2023-07-10    100.95
2023-07-11     42.75
2023-07-12     71.05
2023-07-13     -7.90
2023-07-14   -160.50
2023-07-17   -229.05
2023-07-18      2.85
2023-07-19     82.00
2023-07-20   -111.60
2023-07-21    107.00
2023-07-24    262.75
2023-07-25    123.20
2023-07-26     75.55
2023-07-27    -69.85
2023-07-28    152.40
2023-07-31    136.25
2023-08-01    129.95
2023-08-02   -228.10
2023-08-03    -95.30
2023-08-04   -202.55
2023-08-07    223.85
2023-08-08   -171.00
Name: points, dtype: float64

In [9]:
df_all.set_index('date_open').between_time('10:00', '03:10').groupby('date_only')["points"].sum()

date_only
2023-07-03     61.80
2023-07-04   -350.80
2023-07-05    140.95
2023-07-06    212.65
2023-07-07   -114.35
2023-07-10    181.40
2023-07-11    -26.35
2023-07-12     57.70
2023-07-13     77.70
2023-07-14   -216.50
2023-07-17   -349.20
2023-07-18   -193.15
2023-07-19    112.45
2023-07-20   -153.05
2023-07-21     16.90
2023-07-24    201.55
2023-07-25    123.20
2023-07-26     75.55
2023-07-27    -87.65
2023-07-28    125.85
2023-07-31     80.10
2023-08-01    129.95
2023-08-02   -232.25
2023-08-03   -153.85
2023-08-04   -224.45
2023-08-07    129.85
2023-08-08   -171.00
Name: points, dtype: float64

In [10]:
df_all[df_all['date_only'] == pd.to_datetime('2023-08-08')]

,date_open,Date,open,close,type,points,date_only
339,2023-08-08 10:20:00+05:30,2023-08-08 10:23:00+05:30,44952.30,44995.20,downside,-42.90,2023-08-08
340,2023-08-08 10:24:00+05:30,2023-08-08 10:25:00+05:30,44977.25,45015.85,downside,-38.60,2023-08-08
341,2023-08-08 10:26:00+05:30,2023-08-08 10:34:00+05:30,45003.95,45049.35,downside,-45.40,2023-08-08
342,2023-08-08 10:35:00+05:30,2023-08-08 10:50:00+05:30,45071.60,45044.05,downside,27.55,2023-08-08
343,2023-08-08 11:05:00+05:30,2023-08-08 11:15:00+05:30,44987.90,44946.40,upside,-41.50,2023-08-08
344,2023-08-08 11:16:00+05:30,2023-08-08 11:30:00+05:30,44944.55,44900.65,upside,-43.90,2023-08-08
345,2023-08-08 12:14:00+05:30,2023-08-08 12:25:00+05:30,45016.75,44996.50,downside,20.25,2023-08-08
346,2023-08-08 14:26:00+05:30,2023-08-08 14:37:00+05:30,45031.45,45002.15,downside,29.30,2023-08-08
347,2023-08-08 14:49:00+05:30,2023-08-08 14:50:00+05:30,44929.50,44893.70,upside,-35.80,2023-08-08


## order place

In [5]:
# check the last candle distance 
def buy_sell_order(strike_price = None,ce_pe = None, price = None, trans_type = None,order_type = "MARKET",quantity = 25 ):
    print(strike_price, ce_pe, price,trans_type, order_type)
    order = kite.place_order(
        variety = kite.VARIETY_REGULAR,
        exchange = kite.EXCHANGE_NFO,
        tradingsymbol = f"BANKNIFTY23MAY{strike_price}{ce_pe}",
        transaction_type = trans_type,
        quantity = quantity,
        product = kite.PRODUCT_MIS,
        order_type = order_type,
        price=price,
        validity=kite.VALIDITY_DAY,
        disclosed_quantity=None,
        trigger_price=None,
        squareoff=None,
        stoploss=None,
        trailing_stoploss=None,
        # tag="TradeviaPython",
    )

In [6]:
from util_fun import get_data_parllel
import talib

today_date = '2023-01-01'
start_dt = '2023-05-22'
end_dt = '2023-05-25'
time_frame = 'minute'
instument = 260105

expiry_sp = 'BANKNIFTY23MAY'
inst = pd.DataFrame(kite.instruments("NFO"))

expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [7]:
import time 
# price consolidation i
from IPython.display import clear_output
import time
strike_price = 43800
position = 0
market  = True

if market:
    order_type  = kite.ORDER_TYPE_MARKET
else:
    order_type  = kite.ORDER_TYPE_LIMIT

symbol_ce = f"NFO:BANKNIFTY23MAY{strike_price}CE"
symbol_pe = f"NFO:BANKNIFTY23MAY{strike_price}PE"

up_down = None
position_up = 0
position_down = 0
open_pos_up = 0
close_pos = 0
date_open = 0
sl = 0
df_all  = pd.DataFrame()

while True:
    df_day = get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
    df_day['ema_21'] = talib.EMA(df_day['Close'], timeperiod = 21)
    df_day.dropna(inplace=True)
    df_day['distance'] = (df_day['Close'] - df_day['ema_21'])
    # df_day = df_day.between_time('09:20', '03:20')
    df_day.reset_index(inplace=True)
    last_candle = df_day.tail(1)
    print(last_candle)
    dist_ema = last_candle.distance.values[0]

    if  dist_ema > 0:
        up_down = 1
    else:
        up_down = -1
    # upside
    if dist_ema < -40 and up_down == -1 and position_up == 0:
        # print(c.Date,c.Close,'upside')
        date_open  = last_candle.Date.values[0]
        open_pos = last_candle.Close.values[0]
        position_up = 1 
        sl = last_candle.Close.values[0]  - 30
        ## buy up order
        print('buy up order')
        trans_type = kite.TRANSACTION_TYPE_BUY
        buy_sell_order(strike_price = strike_price, ce_pe = 'CE', trans_type = trans_type,order_type =order_type)
    # downside 
    elif dist_ema > 40 and up_down == 1 and position_down == 0:
        # print(c.Date,c.Close, 'downside')
        date_open  = last_candle.Date.values[0]
        open_pos = last_candle.Close.values[0]
        position_down = 1 
        sl = last_candle.Close.values[0] + 30
        ## buy down order
        print('buy down order')
        trans_type = kite.TRANSACTION_TYPE_BUY
        buy_sell_order(strike_price = strike_price,ce_pe = 'PE', trans_type = trans_type,order_type = order_type)

    # sq of up upside 
    if position_up == 1 and (last_candle.Close.values[0] >= dist_ema or last_candle.Close.values[0] <= sl):
        trans_type = kite.TRANSACTION_TYPE_SELL
        buy_sell_order(strike_price = strike_price, ce_pe = 'CE', trans_type = trans_type,order_type =order_type)

        position_up = 0
        close_pos = last_candle.Close.values[0]
        df = pd.DataFrame({'date_open': date_open, 'Date': last_candle.Date.values[0], 'open':open_pos, 'close': close_pos, 'type':'upside'}, index  = [0] )
        df_all = df_all.append(df)
        # print('sqoff upmove',c.Date,c.Close)

    # sq down side 
    if position_down == 1 and (last_candle.Close.values[0] <= c.dist_ema or last_candle.Close.values[0] >= sl):
        trans_type = kite.TRANSACTION_TYPE_SELL
        buy_sell_order(strike_price = strike_price,ce_pe = 'PE', trans_type = trans_type,order_type = order_type)
        position_down = 0
        close_pos = last_candle.Close.values[0]
        df = pd.DataFrame({'date_open': date_open,'Date': last_candle.Date.values[0],'open':open_pos, 'close': close_pos,'type':'downside'}, index = [0] )
        df_all = df_all.append(df)
        # print('sqoff down move ',c.Date,c.Close)
    time.sleep(20)

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

                         Date      Open     High      Low    Close  Volume  \
990 2023-05-24 13:35:00+05:30  43821.05  43822.7  43818.2  43822.7       0   

           ema_21   distance  
990  43798.262088  24.437912  


Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

                         Date      Open     High      Low    Close  Volume  \
990 2023-05-24 13:35:00+05:30  43821.05  43822.0  43818.2  43821.8       0   

          ema_21  distance  
990  43798.18027  23.61973  


KeyboardInterrupt: 

In [ ]:
df_all = df_all.reset_index(drop = True)
df_all['points'] = np.where(df_all.type == 'upside', df_all['close'] -df_all['open'],df_all['open'] -df_all['close'] )
df_all['date_only'] = df_all.Date.dt.date

df_all.groupby('date_only')["points"].sum()

In [16]:
df_day[(df_day.distance >= 50) & (df_day['Date'].dt.date == pd.to_datetime('2023-05-17'))].head(100).head(50)

,Date,Open,High,Low,Close,Volume,ema_21,distance
4059,2023-05-17 09:25:00+05:30,43924.15,43969.60,43921.10,43964.30,0,43912.475546,51.824454
4328,2023-05-17 13:54:00+05:30,43529.15,43581.65,43529.15,43565.45,0,43509.082594,56.367406
4330,2023-05-17 13:56:00+05:30,43561.00,43571.00,43557.95,43569.05,0,43518.411198,50.638802
4377,2023-05-17 14:43:00+05:30,43568.75,43612.65,43568.75,43608.25,0,43545.980573,62.269427
4378,2023-05-17 14:44:00+05:30,43607.35,43620.05,43606.30,43617.75,0,43552.221393,65.528607
4379,2023-05-17 14:45:00+05:30,43617.95,43665.95,43617.95,43662.15,0,43561.780402,100.369598
4381,2023-05-17 14:47:00+05:30,43618.05,43635.55,43611.85,43631.40,0,43572.123171,59.276829
4382,2023-05-17 14:48:00+05:30,43630.85,43657.50,43630.85,43657.50,0,43579.547243,77.952757
4383,2023-05-17 14:49:00+05:30,43655.50,43663.45,43638.85,43654.65,0,43586.077917,68.572083
4402,2023-05-17 15:08:00+05:30,43661.80,43688.85,43658.40,43685.00,0,43627.033298,57.966702


In [59]:
df_day.describe()

,Open,High,Low,Close,Volume,ema_21,distance
count,1726.000000,1726.000000,1726.000000,1726.000000,1726.0,1726.000000,1726.000000
mean,43882.091222,43892.955272,43870.973696,43881.882503,0.0,43883.293513,-1.411010
std,181.723294,180.749287,182.059467,181.745577,0.0,176.941705,33.104368
min,43447.600000,43459.750000,43447.450000,43448.500000,0.0,43475.352956,-141.978412
25%,43722.600000,43732.275000,43711.712500,43721.600000,0.0,43721.640376,-19.851148
50%,43940.325000,43952.450000,43928.100000,43940.325000,0.0,43940.648670,0.179364
75%,44035.187500,44045.737500,44024.787500,44035.550000,0.0,44036.562217,17.936355
max,44144.900000,44151.700000,44133.650000,44144.600000,0.0,44126.878249,186.020492


In [61]:
pd.qcut(df_day.distance, q=11).reset_index().distance.value_counts()

(-141.979, -45.656]    158
(-17.82, -9.9]         158
(3.336, 9.785]         158
(-45.656, -26.984]     157
(-9.9, -3.299]         157
(9.785, 16.463]        157
(16.463, 24.991]       157
(24.991, 35.374]       157
(35.374, 186.02]       157
(-26.984, -17.82]      156
(-3.299, 3.336]        156
Name: distance, dtype: int64

In [ ]:
# expiry_sp = 'BANKNIFTY23518'
# inst = pd.DataFrame(kite.instruments("NFO"))
# expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# # expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
# expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']